# 实验说明

## 作业说明

### 目标：

训练一个玩2048的神经网络，并得到较高的准确率。

### 背景：

2048是一个益智小游戏，规则为：控制所有方块向同一个方向运动，两个相同数字方块撞在一起后合并，成为他们的和。每次操作时会随机生成一个2或者4，最终得到一个“2048”的方块就算胜利。规则的直观解释：[Click to Play 2048](https://play2048.co/)

本教程展示如何训练一个玩2048的神经网络模型，并评估其最终能够得到的分数。

#### 建模过程：

2048游戏可以理解为一个这样的过程：

<blockquote>
    
有一个**局面（state）**，4x4格子上的一些数字。
    
<img src="https://data.megengine.org.cn/megstudio/images/2048_demo.png" width=256 height=256 />

你可以选择做一些**动作（action）**，比如按键盘上的上/下/左/右键。

你有一些**策略（policy）**，比如你觉得现在按左最好，因为这样有两个8可以合并。对于每个动作，可以用一个打分函数来决定你的策略。

在按照策略做完动作之后，你会得到一个**奖励（reward）**，比如因为两个8合并，分数增加了16，这个16可以被看作是这一步的奖励。

在许多步之后，游戏结束，你会得到一个**回报（return）**，即游戏的最终分数。

</blockquote>

由此，我们将2048建模为一个马尔可夫决策过程，其求解可以通过各种强化学习方法来完成。在baseline中，我们使用了 [Double DQN](https://arxiv.org/abs/1509.06461)。

### 任务：

Q1：训练模型

运行baseline，训练和评估模型。观察游戏结束时的滑动平均分数。你可以调用`print_grid`函数输出模型玩游戏的过程，以判断模型是否可以得到合理的结果。
提供参考数据：纯随机游玩，平均分数约为570分。在baseline的训练过程中，模型最高可以达到8000分，平均为2000分左右。

请你修改参数，模型结构等，使得游戏的平均分数尽可能地高。请注意：这里的平均分数指每个游戏结束**最终分数**的平均值。
**请于q1.diff提交你的代码。**

## 数据集

2048游戏代码来源：[console2048](https://github.com/Mekire/console-2048/blob/master/console2048.py)

## 文件存储
实验中生成的文件可以存储于 workspace 目录中。 查看工作区文件，该目录下的变更将会持久保存。 您的所有项目将共享一个存储空间，请及时清理不必要的文件，避免加载过慢。

## 实验步骤

1.导入库

In [1]:
import megengine as mge
import numpy as np 
import megengine.module as M
import megengine.functional as F
import megengine.data.transform as T
from random import random, randint, shuffle
from megengine.optimizer import Adam
from megengine.autodiff import GradManager
from megengine import tensor
from tqdm import tqdm
import time
import os
import random
import pickle as pickle
import math

2.2048游戏函数

In [2]:
from workspace.backup.game_utils import *

3.定义记忆回放类并实例化

在记录一次决策过程后，我们存储到该类中，并在训练时选择一部分记忆进行训练。

In [3]:
from workspace.backup.perm import rpm, perm

# data = rpm(5000)
data = perm(5000)

4.定义模型结构

In [4]:
from workspace.backup.models import Net, WideNet

model = WideNet()
model_target = WideNet()

5.定义输入转化函数，使得局面可以被输入进模型。

In [5]:
table = {2**i: i for i in range(1, 16)}
table[0] = 0

def make_input(grid):
    # 每个网格对应一个16维向量, 若网格分数为2**i，则向量的第i个分量为1
    g0 = grid
    r = np.zeros(shape=(16, 4, 4), dtype=np.float32)
    for i in range(4):
        for j in range(4):
            v = g0[i, j]
            r[table[v], i, j] = 1
    return r

In [6]:
'''
_mult_step_learning_pri: beta=0.99, epsilon=0.3
_mult_step_learning_pri_2: beta=0.8, epsilon=0.1
'''

'\n_mult_step_learning_pri: beta=0.99, epsilon=0.3\n_mult_step_learning_pri_2: beta=0.8, epsilon=0.1\n'

6.定义优化器

7.模型训练

In [7]:
opt = Adam(model.parameters(), lr=5e-4)
code_label = "_mult_step_learning_pri_2"
batch_size = 512
beta = 0.8 # multi_step weight
epsilon = 0.1
epochs = 10000
dir_path = "workspace/lhw"
model_path = os.path.join(dir_path, f"{code_label}.mge")
loss_path = os.path.join(dir_path, f'all_loss{code_label}.npy')
scores_path = os.path.join(dir_path, f'all_scores{code_label}.npy')
avg_scores_path = os.path.join(dir_path, f'all_avg_scores{code_label}.npy')

maxscore = 0
avg_score = 0

game = []
all_loss = []
all_scores = []
all_avg_scores = []
'''Play 32 games at the same time'''
game = [Game() for _ in range(batch_size)]

In [ ]:
'''
TODO
修改reward, 鼓励最大值放在右下角
'''

In [ ]:
with tqdm(total=epochs*5, desc="epoch") as tq:
    for epoch in range(epochs):

        '''double DQN'''
        if epoch % 10 == 0:  # 每隔10个epoch更新一次target网络

            mge.save(model, model_path)
            model_target = mge.load(model_path)
            np.save(loss_path, np.array(all_loss))
            np.save(scores_path, np.array(all_scores))
            np.save(avg_scores_path, np.array(all_avg_scores))
            if len(all_scores) > 6000:
                break

        grid = []
        for k in range(batch_size):
            '''Check if the game is over'''
            if any_possible_moves(game[k].grid) is False:  # 第k个game游戏结束
                all_scores.append(game[k].score)
                if avg_score == 0:
                    avg_score = game[k].score
                else:
                    avg_score = avg_score * 0.99 + game[k].score * 0.01
                all_avg_scores.append(avg_score)
                game[k] = Game()  # 重新开始一个game

            tmp = make_input(game[k].grid)
            grid.append(tensor(tmp))  # 将所有game的grid转化为状态张量

        status = F.stack(grid, 0)  # status: [b, 16, 4, 4]

        '''Choose the action with the highest probability'''
        a = F.argmax(model(status).detach(), 1)
        a = a.numpy().copy()  # action均使用model 
        for i in range(batch_size):
            if random.random() < epsilon:
                a[i] = random.randint(0, 3)

        if (len(all_scores) > 10000) or (epsilon > 0.01 and epoch % 5 == 0):
            epsilon /= 1.005

        s0_s = []
        s1_s = []
        ak_s = []
        reward_s = []
        done_s = []
        history_reward_s = []

        for k in range(batch_size):
            pre_score = game[k].score
            prev_max = game[k].max()
            pre_grid = game[k].grid.copy()
            empty1 = find_empty_cell(pre_grid)
            game[k].move(a[k])
            after_score = game[k].score
            after_max = game[k].max()
            if game[k].score > maxscore:
                maxscore = game[k].score
            action = a[k]

            '''In some situations, some actions are meaningless, try another'''
            while (game[k].grid == pre_grid).all():
                action = (action + 1) % 4
                game[k].move(action)

            empty2 = find_empty_cell(game[k].grid)
            score = after_score - pre_score

            reward = 0
            if prev_max == after_max:
                reward += math.log2(after_max) * 0.1 # 最大值奖励(只有最大值不改变时才加入?)
            
            reward += (empty2 - empty1) # 空格数增加奖励
            reward += score / 128 # 总得分奖励

            history_reward = game[k].reward
            if history_reward == 0:
                game[k].reward = reward
            else:
                game[k].reward = history_reward * beta + reward * (1 - beta)

            done = any_possible_moves(game[k].grid) is False
            grid = game[k].grid.copy()

            '''Record to memory'''
            '''(status, next_status, action, score, if_game_over)'''
            s0 = tensor(make_input(pre_grid))
            s1 = tensor(make_input(grid))

            s0_s.append(s0)
            s1_s.append(s1)
            ak_s.append(tensor(a[k]))
            reward_s.append(tensor(reward))
            history_reward_s.append(tensor(history_reward))
            done_s.append(tensor(done))

        s0_t = F.stack(s0_s, 0)
        s1_t = F.stack(s1_s, 0)
        #ak_t = F.stack(ak_s, 0)
        #done_t = F.stack(done_s, 0)
        #reward_t = F.stack(reward_s, 0)
        #history_reward_t = F.stack(history_reward_s, 0)
        #print(ak_t.shape)
        cur_vals = model(s0_t).detach()
        #print(cur_vals.shape)
        tar_vals = F.max(model_target(s1_t), axis=1).detach()
        del s0_t, s1_t
        #print(cur_vals.shape, tar_vals.shape)
        #print(cur_vals.shape, tar_vals.shape, done_t.shape, reward_t.shape, history_reward_t.shape)
        #err = F.abs(cur_vals - (tar_vals * 0.99 * (1 - done_t) + reward_t + history_reward_t))
        #print(err.shape)

        for i in range(batch_size):
            #print(type(err[i]), err.shape)
            #print(reward_s[i].shape)
            cur_val = cur_vals[i][ak_s[i]]
            # err = 0
            err = F.abs(cur_val - (tar_vals[i] * 0.99 * (1 - done_s[i]) + reward_s[i] + history_reward_s[i])).numpy().item()
            #print(type(err))
            data.append((s0_s[i], s1_s[i], ak_s[i], reward_s[i], done_s[i], history_reward_s[i]), err)

            '''
            current_value = model(F.expand_dims(s0, 0))[0][a[k]]  
            target_value = F.max(model_target(F.expand_dims(s1, 0)), axis=1)[0]
            #print(type(reward), reward)
            #print(type(history_reward), history_reward)
            #print(type(current_value), current_value)
            #print(type(target_value), target_value)
            err = current_value.item() - (target_value.item() * 0.99 * (1-done) + reward + history_reward)
            error = abs(err)

            data.append((s0, s1, tensor(a[k]), tensor(reward), tensor(done),
                         tensor(history_reward)), error)
            '''

        for i in range(5):
            gm = GradManager().attach(model.parameters())
            with gm:
                res, idxs, weights = data.sample_batch(batch_size)
                s0, s1, a, reward, d, history_reward = res
                # errors = np.empty((0, batch_size), dtype=np.float64)

                '''double DQN'''
                pred_s0 = model(s0)  # (b, 4)
                # (b, 1), target_Q 选择另一张Q值表
                pred_s1 = F.max(model_target(s1), axis=1)

                loss = 0
                total_Q = 0
                total_reward = 0
                total_history_reward = 0
                total_all_reward = 0
                for i in range(batch_size):
                    Q = pred_s0[i][a[i]]  # 预测值 Q(S,a)
                    total_Q += Q
                    total_reward += reward[i]
                    total_history_reward += history_reward[i]
                    tar_val = pred_s1[i].detach() * 0.99 * (1 - d[i]) + reward[i] + history_reward[i]
                    error = F.abs(Q - tar_val)
                    data.update(idxs[i], error.item())

                    loss += float(weights[i]) * F.loss.square_loss(Q, tar_val)

                loss /= batch_size
                total_Q /= batch_size
                total_reward = total_reward / batch_size * 128
                total_history_reward = total_history_reward / batch_size * 128
                total_all_reward = total_reward + total_history_reward
                tq.set_postfix(
                            {
                                "loss": "{0:1.5f}".format(loss.numpy().item()),
                                "Q": "{0:1.5f}".format(total_Q.numpy().item()),
                                "reward": "{0:1.5f}".format(total_reward.numpy().item()),
                                "history_reward": "{0:1.5f}".format(total_history_reward.numpy().item()),
                                "all_reward": "{0:1.5f}".format(total_all_reward.numpy().item()),
                                "avg_score":"{0:1.5f}".format(avg_score),
                                "max_score":"{}".format(maxscore),
                                "epsilon":"{0:1.5f}".format(epsilon),
                                "game_times":"{}".format(len(all_scores)),
                            }
                        )
                tq.update(1)
                all_loss.append(loss.numpy().item())
                gm.backward(loss)

            opt.step()
            opt.clear_grad()

print("maxscore:{}".format(maxscore))
print("avg_score:{}".format(avg_score))

epoch:  25%|██▍       | 12288/50000 [3:37:44<11:00:45,  1.05s/it, loss=2.49913, Q=142.00520, reward=107.50761, history_reward=98.66269, all_reward=206.17030, avg_score=2878.89283, max_score=8488, epsilon=0.00998, game_times=3775] 

In [ ]:
cur_vals[0]
ak_s[0].item()